# Setup

In [15]:
import contracts
contracts.disable_all()

In [16]:
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html

In [17]:
dw.logger.setLevel(50)

Better visualization of output

In [18]:
%%html
<style>
pre {line-height: 90%}
</style>

# Getting data out of the Duckietown World maps
Let's load a map.

In [19]:
m = dw.load_map('ethz_amod_lab_k31')


In [21]:
ipython_draw_html(m);

If you want to get all the tiles, use:

In [22]:
from duckietown_world.geo.measurements_utils import iterate_by_class

records = list(iterate_by_class(m, dw.Tile))

Each "record" here is of class `IterateByTestResult`

In [23]:
records[0]

IterateByTestResult(fqn=('tilemap', 'tile-0-5'), transform_sequence=TransformSequence([Scale2D(scale=0.585), TileCoords(i=0,j=5,orientation=W)]), object=Tile(children={'curve_left': PlacedObject(children={'curve': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],1.5707963267948966))})}, spatial_relations={'curve': GroundTruth(() -> ('curve',)  SE2Transform([0.0, 0.0],4.71238898038469))})}, spatial_relations={1: GroundTruth(() -> ('curve_left',)  SE2Transform([0.0, 0.0],0.0))}), parents=(DuckietownMap(children={'tilemap': TileMap(children={'tile-0-5': Tile(children={'curve_left': PlacedObject(children={'curve': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spa

The record contains the name of the object, the object, and the sequence of transforms:

In [69]:
print('name of object: {}'.format(records[0].fqn))

name of object: ('tilemap', 'tile-0-5')


In [58]:
print('the object: {}'.format(records[0].object))

the object: Tile(children={'curve_left': PlacedObject(children={'curve': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],1.5707963267948966))})}, spatial_relations={'curve': GroundTruth(() -> ('curve',)  SE2Transform([0.0, 0.0],4.71238898038469))})}, spatial_relations={1: GroundTruth(() -> ('curve_left',)  SE2Transform([0.0, 0.0],0.0))})


In [59]:
print('transfoms: {}'.format(records[0].transform_sequence))

transfoms: TransformSequence([Scale2D(scale=0.585), TileCoords(i=0,j=5,orientation=W)])


You probably want to "compress" the transforms into one matrix. For this use the function `asmatrix2d`:

In [27]:
records[0].transform_sequence.asmatrix2d()

Matrix2D(m=[[-5.850000e-01 -7.164184e-17  2.925000e-01]
 [ 7.164184e-17 -5.850000e-01  3.217500e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00]])

## Putting it all together

The following code shows the proper way to "get the tiles with their pose".

Note that each tile is nominally of side 1; the scale is contained in the transformation matrix.

In [54]:
import geometry as geo
import numpy as np

for record in records: # only first 15
    tile = record.object
    matrix = record.transform_sequence.asmatrix2d().m
    translation, angle, scale = geo.translation_angle_scale_from_E2(matrix)
    print('tile kind: %12s   translation: %6.2f %6.2f  angle: %8s deg  scale: %.2f' % (tile.kind, translation[0], translation[1], np.rad2deg(angle), scale))

tile kind:   curve_left   translation:   0.29   3.22  angle:   -180.0 deg  scale: 0.58
tile kind:     straight   translation:   0.88   3.22  angle:   -180.0 deg  scale: 0.58
tile kind:    3way_left   translation:   1.46   3.22  angle:   -180.0 deg  scale: 0.58
tile kind:     straight   translation:   2.05   3.22  angle:   -180.0 deg  scale: 0.58
tile kind:   curve_left   translation:   2.63   3.22  angle:     90.0 deg  scale: 0.58
tile kind:      asphalt   translation:   3.22   3.22  angle:      0.0 deg  scale: 0.58
tile kind:     straight   translation:   0.29   2.63  angle: -90.00000000000001 deg  scale: 0.58
tile kind:      asphalt   translation:   0.88   2.63  angle:      0.0 deg  scale: 0.58
tile kind:     straight   translation:   1.46   2.63  angle: -90.00000000000001 deg  scale: 0.58
tile kind:      asphalt   translation:   2.05   2.63  angle:      0.0 deg  scale: 0.58
tile kind:   curve_left   translation:   2.63   2.63  angle: -90.00000000000001 deg  scale: 0.58
tile kind:   

# Getting other objects

The same can be repeated with lane segments, traffic signs, etc.

For example, the following iterates over traffic signs:



In [74]:
import geometry as geo
import numpy as np

for record in list(iterate_by_class(m, dw.Tile)):
    print(ob.children)
    matrix = record.transform_sequence.asmatrix2d().m
    translation, angle, scale = geo.translation_angle_scale_from_E2(matrix)
    print('object: %22s   translation: %6.2f %6.2f  angle: %8s deg  scale: %.2f' % (ob, translation[0], translation[1], np.rad2deg(angle), scale))

{}
object: SignRightTIntersect(children={}, spatial_relations={})   translation:   0.29   3.22  angle:   -180.0 deg  scale: 0.58
{}
object: SignRightTIntersect(children={}, spatial_relations={})   translation:   0.88   3.22  angle:   -180.0 deg  scale: 0.58
{}
object: SignRightTIntersect(children={}, spatial_relations={})   translation:   1.46   3.22  angle:   -180.0 deg  scale: 0.58
{}
object: SignRightTIntersect(children={}, spatial_relations={})   translation:   2.05   3.22  angle:   -180.0 deg  scale: 0.58
{}
object: SignRightTIntersect(children={}, spatial_relations={})   translation:   2.63   3.22  angle:     90.0 deg  scale: 0.58
{}
object: SignRightTIntersect(children={}, spatial_relations={})   translation:   3.22   3.22  angle:      0.0 deg  scale: 0.58
{}
object: SignRightTIntersect(children={}, spatial_relations={})   translation:   0.29   2.63  angle: -90.00000000000001 deg  scale: 0.58
{}
object: SignRightTIntersect(children={}, spatial_relations={})   translation:   0.88

In [71]:
[k for k, val in ob.__dict__.items() if not str(hex(id(val))) in str(val)]

['children', 'spatial_relations', 'tag']